Task: Fine-tune an open source LLM for domain name suggestions. Additionally, model must refuse to product inappropriate/harmful content domain names.

Part I: Synthetic Dataset Creation

We will start by asking Chatgpt to create domain names and ask whether or not they are invalid or valid (based on whether or not they are appropriate)

In [ ]:
!pip install openpyxl

In [1]:
import pandas as pd

# Read the Excel file
df = pd.read_excel('data/Business_descriptions.xlsx')

# Check number of valid or invalid
print("Distribution of Validity column:")
print(df['Validity'].value_counts())

# Display the count of unique Business Descriptions
print("\nCount of unique Business Descriptions:")
print(df['Business Description'].nunique())

df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'data/Business_descriptions.xlsx'

Let's make sure that we only grab the unique business descriptions.

In [ ]:
from sklearn.model_selection import train_test_split

# Drop duplicate business descriptions and create a copy
df = df.drop_duplicates(subset=['Business Description']).copy()

# Create a new column 'label' with 1 for 'Valid' and 0 for 'Invalid'
df['label'] = df['Validity'].apply(lambda x: 1 if x == 'Valid' else 0)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    df['Business Description'],
    df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# Display the shapes of the training and validation sets
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)

Training set shape: (468,)
Validation set shape: (118,)


In [ ]:
!pip install transformers==4.34.1 bitsandbytes==0.41.1 accelerate

INFO: pip is looking at multiple versions of accelerate to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of accelerate to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
   --------------------- ------------------ 4.2/7.7 MB 20.9 MB/s eta 0:00:01
   ---------------------------------------- 7.7/7.7 MB 21.8 MB/s  0:00:00
   ---------------------------------------- 0.0/92.6 MB ? eta -:--:--
   -- ------------------------------------- 5.2/92.6 MB 31.9 MB/s eta 0:00:03
   ---- ----------------------------------- 10.7/92.6 MB 28.0 MB/s eta 0:00:03
   ------- 

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch, json, pandas as pd, re

MODEL_ID = "openai/gpt-oss-20b"
USE_4BIT = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=USE_4BIT,
)

judge = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.0,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


AttributeError: 'BitsAndBytesConfig' object has no attribute 'get_loading_attributes'

In [3]:
import os

print("Files in the current directory:")
print(os.listdir('.'))
print("\nFiles in the 'data' directory (if it exists):")
try:
    print(os.listdir('data'))
except FileNotFoundError:
    print("'data' directory not found.")

Files in the current directory:
['.config', 'sample_data']

Files in the 'data' directory (if it exists):
'data' directory not found.
